In [1]:
from pynestml.frontend.pynestml_frontend import generate_nest_target

nestml_file_path = "/home/shimoura/Documents/GitHub/motor-controller-model/nestml-neurons/"
nestml_target_path = nestml_file_path + "nestml_target"

generate_nest_target(input_path=nestml_file_path,
                     target_path=nestml_target_path,)


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.7.0
 Built: May 24 2024 10:11:53

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.7.0
 Built: May 24 2024 10:11:53

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.

CMake Warning:
  Ignoring empty string ("") provided on the command line.


-- The CXX compiler identification is GNU 13.3.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /home/shimoura/miniforge3/envs/motor-controller/bin/x86_64-conda-linux-gnu-c++ - skipped
-- Detecting C

In [2]:
import nest
nest.Install("nestmlmodule")


Jun 24 17:57:19 Install [Info]: 
    loaded module nestmlmodule


In [3]:
input_neuron = nest.Create("rb_neuron")
output_neuron = nest.Create("basic_neuron")